In [ ]:
import os
import cv2
import json
import torch
import numpy as np
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import matplotlib.pyplot as plt
from tkinter import Tk, filedialog
import time

In [8]:
############################################
# 1️⃣ Model Paths (Windows)
############################################

food_model_dir = r"D:\Food\food_type_model"
shape_model_dir = r"D:\Food\shape_model"


In [9]:
############################################
# 2️⃣ Load Model Function
############################################

def load_model(model_dir, dataset_name):
    cfg = get_cfg()
    cfg.merge_from_file(os.path.join(model_dir, "config.yaml"))
    cfg.MODEL.WEIGHTS = os.path.join(model_dir, "model_final.pth")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    with open(os.path.join(model_dir, "metadata.json"), "r") as f:
        metadata_dict = json.load(f)

    metadata = MetadataCatalog.get(dataset_name)
    metadata.set(thing_classes=metadata_dict["thing_classes"])

    predictor = DefaultPredictor(cfg)
    return predictor, metadata

In [ ]:
############################################
# 3️⃣ Load Both Models
############################################

print("Loading food model...")
food_predictor, food_metadata = load_model(food_model_dir, "food_dataset")

print("Loading shape model...")
shape_predictor, shape_metadata = load_model(shape_model_dir, "shape_dataset")

print("✅ Both models loaded successfully!")

############################################
# 4️⃣ Upload Image (File Dialog)
############################################

Tk().withdraw()
image_path = filedialog.askopenfilename(title="Select an Image")

if not image_path:
    print("No image selected.")
    exit()

image = cv2.imread(image_path)


Loading food model...
Loading shape model...
✅ Both models loaded successfully!


In [ ]:
############################################
# 5️⃣ Run Both Models
############################################

food_outputs = food_predictor(image)
shape_outputs = shape_predictor(image)

############################################
# 6️⃣ Visualize Results
############################################

v1 = Visualizer(image[:, :, ::-1], metadata=food_metadata, scale=1.0)
food_vis = v1.draw_instance_predictions(food_outputs["instances"].to("cpu"))

v2 = Visualizer(image[:, :, ::-1], metadata=shape_metadata, scale=1.0)
shape_vis = v2.draw_instance_predictions(shape_outputs["instances"].to("cpu"))

plt.figure(figsize=(15,6))

plt.subplot(1,2,1)
plt.title("Food Detection")
plt.imshow(food_vis.get_image())

plt.subplot(1,2,2)
plt.title("Shape Detection")
plt.imshow(shape_vis.get_image())

plt.show()

In [ ]:
############################################
# 7️⃣ Extract Bounding Boxes
############################################

food_instances = food_outputs["instances"].to("cpu")
shape_instances = shape_outputs["instances"].to("cpu")

food_boxes = food_instances.pred_boxes.tensor.numpy()
food_classes = food_instances.pred_classes.numpy()

shape_boxes = shape_instances.pred_boxes.tensor.numpy()
shape_classes = shape_instances.pred_classes.numpy()


In [ ]:
############################################
# 8️⃣ IoU Function
############################################

def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    return interArea / float(boxAArea + boxBArea - interArea + 1e-6)

In [ ]:
############################################
# 9️⃣ Compare Overlaps
############################################

print("\n🔍 Mapping Food Items to Shapes:\n")

for i, food_box in enumerate(food_boxes):
    food_label = food_metadata.thing_classes[food_classes[i]]

    for j, shape_box in enumerate(shape_boxes):
        shape_label = shape_metadata.thing_classes[shape_classes[j]]

        iou = compute_iou(food_box, shape_box)

        if iou > 0.3:
            print(f"✅ {food_label} belongs to {shape_label} (IoU={iou:.2f})")